# TruBot Populate DB

This notebook tests a conversation bot with vector-store context of TruEra website. The database is reset and several pre-defined queries are made to test the four chain variants.

In [ ]:
# Execute this cell if running from github repo.

%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.parent.parent.resolve()))

In [ ]:
from trulens.core.utils.keys import check_or_set_keys

check_or_set_keys(
    OPENAI_API_KEY="to fill in",
    HUGGINGFACE_API_KEY="to fill in",
    PINECONE_API_KEY="to fill in",
    PINECONE_ENV="to fill in",
)

In [ ]:
from pprint import PrettyPrinter

from trulens.core import Tru
from trulens.core.utils.threading import TP

pp = PrettyPrinter()

# Reset the database if needed:
Tru().reset_database()

In [ ]:
from examples.expositional.end2end_apps.trubot.trubot import get_answer
from examples.expositional.end2end_apps.trubot.trubot import get_or_make_app

app = get_or_make_app(cid=None, selector=3)

In [ ]:
# List the app components that got instrumented along with their categories.

app.print_instrumented()

In [ ]:
proc = Tru().start_dashboard(
    force=True, _dev=Path.cwd().parent.parent.parent.parent
)

In [ ]:
from trulens.core.schema import FeedbackMode

selectors = [0, 1, 3, 4]
messages = [
    "Who is Shayak?",
    "Wer ist Shayak?",
    "Kim jest Shayak?",
    "¿Quién es Shayak?",
    "Was ist QII?",
    "Co jest QII?",
]

# Comment this out to run all chain variants and all test queries:
selectors = selectors[0:1]
messages = messages[0:1]


def test_bot(selector, question):
    print(selector, question)
    app = get_or_make_app(
        cid=question + str(selector),
        selector=selector,
        feedback_mode=FeedbackMode.DEFERRED,
    )
    answer = get_answer(app=app, question=question)
    return answer


results = []

for s in selectors:
    for m in messages:
        results.append(TP().submit(test_bot, selector=s, question=m))

In [ ]:
thread = Tru().start_evaluator(restart=True)